In [136]:
import requests
import json
from bs4 import BeautifulSoup
import re
from datetime import datetime

import pandas as pd

import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [137]:
# driver = webdriver.Chrome()

In [233]:
# url = 'https://www.sec.gov/Archives/edgar/data/0001052837/000104746905006618/a2153526z10-k.htm'
# url = 'https://www.sec.gov/Archives/edgar/data/0000356809/000095013309001161/w73596e10vk.htm'
# url = 'https://www.sec.gov/edgar/browse/?CIK=1134538'
# url = 'https://www.sec.gov/Archives/edgar/data/0000738076/000095013507004539/b659553ce10vk.htm#302'
# url = 'https://www.sec.gov/Archives/edgar/data/0000001923/000095014402007559/g77316e10vk.htm'
# url = 'https://www.sec.gov/Archives/edgar/data/0000096638/000089322006000479/w18203e10vk.htm#tocpage'
# url = 'https://www.sec.gov/Archives/edgar/data/0000738076/000095012309025431/b76396cce10vk.htm'
# url = 'https://www.sec.gov/Archives/edgar/data/0000930095/000114544304000408/d14254.htm'
url = 'https://www.sec.gov/Archives/edgar/data/0001068887/000095014402010112/g78526e10vk.htm'
# url = 'https://www.sec.gov/Archives/edgar/data/0001010775/000110465906016299/a06-1906_110k.htm'
headers = {"User-Agent": "Anselme F.E. Borgeaud (aborgeaud@gmail.com)"}

page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

# soup.title
#
# driver.get(url)

In [209]:
# search_elem = driver.find_element_by_xpath(
#             '//input[@placeholder="Search table"]'
#         )
# search_elem.send_keys("10-K ")

In [212]:
# date_strings = driver.find_elements_by_xpath('//a[@data-index="reportDate"]')
# for date_str in date_strings:
#     print(date_str.text)

2008-08-31
2007-08-31
2006-08-31
2005-08-31
2004-08-31
2003-08-31
2002-08-31
2001-08-31


In [139]:
def is_balance_sheet(table):
    content = table.text.lower()
    has_assets = (len(re.findall(r"current.*assets", content, re.DOTALL)) > 0) or (
        len(re.findall(r"total.*assets", content, re.DOTALL)) > 0
    )
    has_liab = (len(re.findall(r"total.*liabilities", content, re.DOTALL)) > 0) or (
        len(re.findall(r"current.*liabilities", content, re.DOTALL)) > 0
    )
    has_cash = "cash" in content
    has_equity = "equity" in content
    has_asset = "assets" in content
    return has_asset and has_assets and has_liab and has_cash and has_equity
        
    

def get_balance_table(soup):
    bs = soup.findAll("b")
    b_balance = [b for b in bs if 'consolidated balance sheet' in b.text.lower()][0]
    parent = b_balance.find_parent('div')
    i = 0
    table = None
    while (table is None) and (i < 4):
        parent = parent.find_next_sibling('div')
        table = parent.find('table')
        i += 1
    
    try:
        return pd.read_html(table.prettify(), flavor='bs4')[0]
    except:
        return None
    
def get_balance_table_2(soup):
    tables = soup.findAll("table")
    balance_tables = [t for t in tables if is_balance_sheet(t)]
    if len(balance_tables) == 0:
        return None
    i_longest = 0
    longest = 0
    for i, table in enumerate(balance_tables):
        if len(table) > longest:
            i_longest = i
            longest = len(table)
    balance_table = balance_tables[i_longest]
    dfs = pd.read_html(balance_table.prettify(), flavor="bs4")
    if len(dfs) == 1:
        return dfs[0]

def process_balance(df):
    i, index_col = find_index_col(df)
    arr = df.to_numpy()[:, i:]
    mask = [type(x) == str for x in arr[:, 0]]
    arr = arr[mask, :]
    data = np.apply_along_axis(lambda x: get_value(x), 1, arr)
    index = arr[:, 0]
    df = pd.DataFrame({'value': data}, index=index)
    return df

def get_value(x):
    vals = [v for v in x if is_finite(v)]
    try:
        val = float(vals[0]) * 1000
    except:
        return None
    return val

def is_finite(x):
    try:
        f = float(x)
        return np.isfinite(f)
    except:
        return False
    
def find_index_col(df):
    index_col = ""
    i_index_col = 0
    for i, col in enumerate(df.columns):
        if df.loc[:, col].astype("str").str.contains("assets").any():
            index_col = col
            i_index_col = i
    return i_index_col, index_col

In [141]:
# bs = soup.findAll("b")
# b_balance = [b for b in bs if 'consolidated balance sheet' in b.text.lower()][0]
# parent = b_balance.find_parent('div')
# # balance_table = parent.find_next_sibling('div').find_next_sibling('div').find('table')
# parent.find_next_sibling('div').find('table')

IndexError: list index out of range

In [33]:
pd.read_html(balance_table.prettify(), flavor='bs4')[0]

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,"May 31,","May 31,","May 31,","May 31,","May 31,","May 31,",NaN
2,NaN,NaN,2007,2007,NaN,NaN,2006,2006,NaN
3,ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Current assets:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cash and equivalents,NaN,$,559217,NaN,NaN,$,501097,NaN
6,Short-term investments,NaN,NaN,—,NaN,NaN,NaN,363250,NaN
7,Notes receivable,NaN,NaN,77368,NaN,NaN,NaN,63224,NaN
8,"Accounts receivable, less allowance for doubtf...",NaN,NaN,102952,NaN,NaN,NaN,115120,NaN
9,Inventories,NaN,NaN,107988,NaN,NaN,NaN,148819,NaN


In [98]:
df = get_balance_table(soup)
i, index_col = find_index_col(df)
arr = df.to_numpy()[:, i:]
mask = [type(x) == str for x in arr[:, 0]]
arr = arr[mask, :]
np.apply_along_axis(lambda x: get_value_2(x), 1, arr)

array([None, None, 559217000.0, 363250000.0, 77368000.0, 102952000.0,
       107988000.0, 50157000.0, 897682000.0, 76460000.0, 766444000.0,
       371289000.0, 39217000.0, 2151092000.0, None, None, 110430000.0,
       94000000.0, 435638000.0, 640068000.0, 23725000.0, 336000000.0,
       173930000.0, None, None, 2323356000.0, None, None, 4349000.0,
       1151299000.0, 2151092000.0], dtype=object)

In [105]:
data = np.apply_along_axis(lambda x: get_value_2(x), 1, arr)
index = arr[:, 0]
df = pd.DataFrame({'value': data}, index=index)

In [113]:
df = get_balance_table(soup)
df_proc = process_balance(df)

In [114]:
df_proc.transpose()

,ASSETS,Current assets:,Cash and equivalents,Short-term investments,Notes receivable,"Accounts receivable, less allowance for doubtful accounts of $15,292 and $16,422, respectively",Inventories,Other current assets,Total current assets,"Property and equipment, less accumulated depreciation and amortization of $234,554 and $232,944, respectively",...,Long-term debt,Minority interest,Stockholders’ equity:,"Preferred stock, $0.01 par value, 10,000 shares authorized; none outstanding","Common stock, $0.01 par value, 990,000 shares authorized; shares issued: 399,064 and 393,442 respectively",Unamortized stock-based compensation,Retained deficit,Accumulated other comprehensive income (loss),Total stockholders’ equity,Total liabilities and stockholders’ equity
value,None,None,559217000.0,363250000.0,77368000.0,102952000.0,107988000.0,50157000.0,897682000.0,76460000.0,...,336000000.0,173930000.0,None,None,2323356000.0,None,None,4349000.0,1151299000.0,2151092000.0


In [35]:
get_date(soup)

In [36]:
bs = soup.findAll("b")
texts = [
        p.get_text(strip=True).replace(u"\xa0", u" ")
        for p in bs
        if "year ended" in p.text
    ]
texts

['The Company experienced a net loss for the year ended December 31, 2008, the first net loss in\nthe history of the Company.']

In [37]:
bs

[<b>UNITED STATES<br/>
 SECURITIES AND EXCHANGE COMMISSION</b>,
 <b>Washington, D.C. 20549</b>,
 <b>FORM 10-K</b>,
 <b>ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934</b>,
 <b><b>For the fiscal ended December 31, 2008.</b>
 </b>,
 <b>For the fiscal ended December 31, 2008.</b>,
 <b>or</b>,
 <b>TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934<br/></b>,
 <b><b>For the transition period from <u>                    </u> to <u>                    </u>.</b>
 </b>,
 <b>For the transition period from <u>                    </u> to <u>                    </u>.</b>,
 <b>Commission file number: <u>000-10971</u></b>,
 <b><u> ABIGAIL ADAMS NATIONAL BANCORP, INC.</u></b>,
 <b>Delaware</b>,
 <b>52-1508198</b>,
 <b>1130 Connecticut Avenue, NW <br/>
 Washington, DC</b>,
 <b>20036</b>,
 <b>(202) 772-3600</b>,
 <b>PART I</b>,
 <b>Item 1. Business.</b>,
 <b>General</b>,
 <b>Recent Developments</b>,
 <b>Recent Market Developments<

In [18]:
match = re.findall(r"ended .*[0-9]{4}", texts[0])
match

['ended December 31, 2004']

In [20]:
date_str = match[0].replace("ended ", "")
date_str

'December 31, 2004'

In [23]:
date = datetime.strptime(date_str, "%B %d, %Y")
date

datetime.datetime(2004, 12, 31, 0, 0)

In [152]:
url = 'https://www.sec.gov/Archives/edgar/data/0001065728/000136110608000452/able_10k-063008.htm'
headers = {"User-Agent": "Anselme F.E. Borgeaud (aborgeaud@gmail.com)"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [125]:
fonts = soup.findAll("font")
fonts

[<font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">UNITED
 STATES</font>,
 <font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">SECURITIES
 AND EXCHANGE COMMISSION</font>,
 <font style="DISPLAY: inline; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">Washington,
 D.C. 20549</font>,
 <font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">FORM
 10-K</font>,
 <font style="DISPLAY: inline; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">(Mark
 One)</font>,
 <font face="Wingdings" style="DISPLAY: inline;">x</font>,
 <font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">ANNUAL
 REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE</font>,
 <font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE: 10pt; FONT-FAMILY: Times New Roman">SECURITIES
 EXCHANGE ACT OF 1934</font>,
 <font style="DISPLAY: inline; FONT-WEIGHT: bold; FONT-SIZE

In [134]:
tables = soup.findAll("table")
len(tables)

193

In [148]:
df_balance = pd.read_html(balance_table.prettify(), flavor='bs4')[0]
process_balance(df_balance)

,value
ASSETS,None
Current Assets:,None
Cash and cash equivalents,3025577000.0
"Accounts receivable, net of allowance for doubtful accounts of",None
"$1,283,013 and $744,253, at June 30, 2008 and 2007, respectively",4088377000.0
Inventories,2724315000.0
Notes receivable - current portion,725000000.0
Advances to related parties,8374496000.0
Prepaid expenses and other current assets,1676658000.0
Total Current Assets,11514927000.0


In [142]:
df = get_balance_table_2(soup)

In [149]:
tables = soup.findAll("table")
balance_tables = [t for t in tables if is_balance_sheet(t)]
i_longest = 0
longest = 0
for i, table in enumerate(balance_tables):
    if len(table) > longest:
        i_longest = i
        longest = len(table)
balance_table = balance_tables[i_longest]

<table border="0" cellpadding="0" cellspacing="0" style="border-collapse:collapse;">
<tr>
<td style="padding:0pt .7pt 0pt 0pt;width:343.35pt;" valign="bottom" width="458">
<p style="font-weight:bold;line-height:7.0pt;margin:0pt 0pt .0001pt;page-break-after:avoid;text-align:center;"><b><font face="Times New Roman" size="1" style="font-size:1.0pt;"> </font></b></p>
</td>
<td style="padding:0pt .7pt 0pt 0pt;width:12.0pt;" valign="bottom" width="16">
<p style="font-weight:bold;line-height:7.0pt;margin:0pt 0pt .0001pt;page-break-after:avoid;text-align:center;"><b><font face="Times New Roman" size="1" style="font-size:1.0pt;"> </font></b></p>
</td>
<td colspan="8" style="border:none;border-bottom:solid windowtext 1.0pt;padding:0pt .7pt 0pt 0pt;width:96.2pt;" valign="bottom" width="128">
<p style="font-weight:bold;line-height:7.0pt;margin:0pt 0pt .0001pt;page-break-after:avoid;text-align:center;"><b><font face="Times New Roman" size="1" style="font-size:7.0pt;">For the Years Ended<br/>
  Dece

In [189]:
tables = soup.findAll("table")
# balance_tables = [t for t in tables if is_balance_sheet(t)]
for i,table in enumerate(tables):
    content = table.text.lower()
    has_assets = (len(re.findall(r"current.*assets", content, re.DOTALL)) > 0) or (
        len(re.findall(r"total.*assets", content, re.DOTALL)) > 0
    )
    has_liab = (len(re.findall(r"total.*liabilities", content, re.DOTALL)) > 0) or (
        len(re.findall(r"current.*liabilities", content, re.DOTALL)) > 0
    )
    has_cash = "cash" in content
    has_equity = "equity" in content
    has_asset = "assets" in content
#     print(i, has_asset, has_assets , has_liab , has_cash , has_equity)

# print(tables[68].prettify())


'current assets' in tables[74].text.lower()
'liabilities' in ' '.join([e.text.lower() for e in tables[51].findAll('td')])
# s = tables[74].text.lower()
# print(s)

# re.findall(r'total.*liabilities', s, re.DOTALL)

False

In [134]:
process_balance(pd.read_html(tables[68].prettify(), flavor='bs4')[0])

,value
LIABILITIES AND STOCKHOLDERS’ EQUITY,None
Current liabilities:,None
Long-term debt,224688000.0
Deferred income taxes,2122000.0
Minority interest in consolidated joint venture,1102000.0
Stockholders’ equity:,None


In [7]:
balance_sheet = get_balance_table_2(soup)
balance_sheet = process_balance(balance_sheet)

IndexError: list index out of range

In [205]:
balance_sheet

,value
ASSETS,None
Current assets:,None
Cash and equivalents,545818000.0
Short-term investments,98357000.0
Notes receivable,40590000.0
"Accounts receivable, less allowance for doubtful accounts of $9,645 and $12,253, respectively",112771000.0
Inventories,90395000.0
Other current assets,56982000.0
Total current assets,944913000.0
"Property and equipment, less accumulated depreciation and amortization of $172,717 and $205,835, respectively",40012000.0


In [206]:
date = '2021-01-01'
cik = 12345
add_info = pd.DataFrame(
                data={'value': [date, cik, page.url]}, index=["date_filled", "CIK", "url"]
            )
balance_sheet = balance_sheet.append(add_info)
balance_sheet = balance_sheet.transpose().reset_index(drop=True)
balance_sheet

,ASSETS,Current assets:,Cash and equivalents,Short-term investments,Notes receivable,"Accounts receivable, less allowance for doubtful accounts of $9,645 and $12,253, respectively",Inventories,Other current assets,Total current assets,"Property and equipment, less accumulated depreciation and amortization of $172,717 and $205,835, respectively",...,Stockholders’ equity:,"Preferred stock, $0.01 par value, 10,000 shares authorized; none outstanding","Common stock, $0.01 par value, 990,000 shares authorized; shares issued and outstanding: 389,284 and 405,656, respectively",Retained deficit,Accumulated other comprehensive income,Total stockholders’ equity,Total liabilities and stockholders’ equity,date_filled,CIK,url
0,None,None,545818000.0,98357000.0,40590000.0,112771000.0,90395000.0,56982000.0,944913000.0,40012000.0,...,None,None,2336961000.0,None,65736000.0,1112175000.0,1815357000.0,2021-01-01,12345,https://www.sec.gov/Archives/edgar/data/000073...


In [235]:
def find_value(row_elem):
        def is_number(s):
            return len(re.findall(r'[0-9]+', s)) > 0

        elem = row_elem.find_next_sibling('td')
        if elem is None:
            return None
        i = 0
        while not is_number(elem.text) and i < 4:
            elem = elem.find_next_sibling('td')
        if is_number(elem.text):
            try:
                return float(elem.text.replace(',', ''))
            except ValueError:
                return None

def find_value_in_table(soup, key: str) -> float:
    key_elem = None
    key_text = key.strip().replace('\n', '_').replace(' ', '_').lower()
    for e in soup.findAll('td'):
        text = e.text.strip().replace('\n', '_').replace(' ', '_').lower()
        if key_text == text:
            key_elem = e
    if key_elem:
        return find_value(key_elem)

    
print(find_value_in_table(soup, 'total_current_assets'))

276106.0
